In [6]:
#how decorators work
#addeding a decorator funtion that takes the base funtion as an argument
def add_sprinkles(func):
    def wrapper(*args, **kwargs):#definnig a wrapper function tha takes in any number of arguments
        print("Adds sprinkles... ")
        func(*args, **kwargs)#calling the base funtion
    return wrapper#returning the wrapper funtion ie the base function and the extended functionality/decorator

def add_fudge(func):
    def wrapper(*args, **kwargs):
        print("Adds fudge...")
        func(*args, **kwargs)
    return wrapper
    
#defining a base funtion
@add_fudge#calling the decorator function
@add_sprinkles
def print_ice_cream(flavor):
    print(f"Here is a {flavor} ice cream")

print_ice_cream("vanilla")#calling the base funtion

Adds fudge...
Adds sprinkles... 
Here is a vanilla ice cream


In [1]:
def my_map(my_func, list):
    result = []
    for item in list:
        new_item = my_func(item)
        result.append(new_item)
    return result


num = [3, 4, 5 , 6, 7]
print(my_map((lambda x: x**3), num))

[27, 64, 125, 216, 343]


In [ ]:
from experta import *
from collections import defaultdict

class Rule:
    def __init__(self, *args):
        self.conditions = []
        for arg in args:
            if isinstance(arg, Fact):
                self.conditions.append(arg)
            elif callable(arg):
                self.conditions.append(arg)

    def __call__(self, engine):
        """
        This method is called when the rule is evaluated.
        It checks if all conditions are met and executes the associated action.
        """
        for condition in self.conditions:
            if callable(condition):
                if not condition(engine):
                    return False  # Condition not met
            elif not engine.facts.match(condition):
                return False  # Fact not found

        # All conditions met, execute the action
        engine.agenda.append(self)
        return True

class KnowledgeEngine:
    def __init__(self):
        self.facts = defaultdict(list)
        self.agenda = []

    def declare(self, fact):
        if isinstance(fact, Fact):
            self.facts[type(fact)].append(fact)
        else:
            raise TypeError("Only Fact objects can be declared.")

    def run(self):
        while self.agenda:
            rule = self.agenda.pop(0)
            rule.execute(self) 

# Example Usage
class MyKnowledgeBase(KnowledgeEngine):
    @Rule(Fact('animal', family='felinae'))
    def is_a_cat(self):
        print("This is a cat!")

engine = MyKnowledgeBase()
engine.reset()
engine.declare(Fact('animal', family='felinae'))
engine.run()

This is a cat!
This is a cat!


In [ ]:
from experta import *

class diagoseEngine(KnowledgeEngine):
    @Rule(Fact(symptom ='fever'))
    def malaria(self):
        print("You have malaria")

    @Rule(Fact(symptom = 'cough'))
    def flu(self):
        print("You have a flu")
    
    @Rule(Fact(symptom = 'headache'))
    def hypertension(self):
        print("you have hypertension")
    
    #adding a wildcard rule
    @Rule(NOT(Fact(symptom = W())))
    def dont_know(self):
        print("I dont know what you have")  
    

engine = diagoseEngine()
engine.reset()

symptoms = input("enter a list of your symptoms comma seperated")
engineInput = symptoms.lower().split(",")
for s in engineInput:
    engine.declare(Fact(symptom = s))

engine.run()cough_

In [ ]:
class KnowledgeBase:
    def __init__(self):
        self.diseases = {
            "Common Cold": {"symptoms": ["cough", "sneezing", "sore throat"], "treatment": "rest and hydration"},
            "Flu": {"symptoms": ["fever", "fatigue", "body aches"], "treatment": "antiviral medication"},
            "Pneumonia": {"symptoms": ["chest pain", "cough", "fever"], "treatment": "antibiotics"},
            "Tuberculosis": {"symptoms": ["cough with blood", "fatigue", "weight loss"], "treatment": "antibiotics"},
            "Allergies": {"symptoms": ["sneezing", "itchy eyes", "rash"], "treatment": "antihistamines"}
        }

        self.relationships = {
            "common cold": ["flu", "allergies"],
            "flu": ["common cold", "pneumonia"],
            "pneumonia": ["flu"],
            "tuberculosis": [],
            "allergies": ["common cold"],
            "cough": ["common cold", "flu", "pneumonia", "tuberculosis"],
            "sneezing": ["common cold", "flu", "allergies"],
            "fever": ["flu", "pneumonia"]

        }

    def diagnose(self, symptoms):
        possible_diseases = {}
        for symptom in symptoms:
            for disease in self.relationships.get(symptom, []):
                possible_diseases[disease] = possible_diseases.get(disease, 0) + 1

        # Sort diseases by the number of matching symptoms (most relevant first)
        sorted_diseases = sorted(possible_diseases.items(), key=lambda x: x[1], reverse=True)

        return [disease for disease, _ in sorted_diseases]

# Interactive mode
kb = KnowledgeBase()
print("Welcome to the Medical Diagnosis Knowledge-Based System")
print("Possible symptoms:", ", ".join(kb.relationships.keys()))
print("\nEnter symptoms (comma separated):")
user_input = input(">").lower().split(",")

symptoms = [symptom.strip() for symptom in user_input]
diagnoses = kb.diagnose(symptoms)

if diagnoses:
    print("Possible diagnoses:")
    for disease in diagnoses:
        print(f"- {disease}: {kb.diseases[disease]['treatment']}")
else:
    print("No diagnoses found. Please consult a doctor.")

Welcome to the Medical Diagnosis Knowledge-Based System
Possible symptoms: common cold, flu, pneumonia, tuberculosis, allergies, cough, sneezing, fever

Enter symptoms (comma separated):
No diagnoses found. Please consult a doctor.
